In [ ]:
from datetime import date

import pandas as pd

# parse_dates 可以直接解析date
df = pd.read_excel("/data/tmp/2023.09_中上协_董事兼职情况_(2022).xlsx", sheet_name=1,
                   parse_dates=['start_date', 'end_date'])
df.head()

In [ ]:
# 去重
df = df.drop_duplicates(subset=["people_name", "comp_name_zh_cn", "employer"])
df

In [ ]:
# 删除无效数据
df = df.dropna(subset=["people_name", "comp_name_zh_cn", "employer"])
df

In [ ]:
# 筛选出 'start_date' 长度小于 6 的行
# 当组合多个条件语句时，每个条件都必须用括号括起来()。而且，不能使用 or/ and，而需要使用or运算符|和and 运算符&。
filtered_rows = df[(df['start_date'].str.len() < 10) | (df['end_date'].str.len() < 10)]
filtered_rows['start_date'] = filtered_rows['start_date'].apply(lambda x: (str(x) + '-01' * 2)[:10])
filtered_rows['end_date'] = filtered_rows['end_date'].apply(lambda x: (str(x) + '-01' * 2)[:10])
filtered_rows

In [ ]:
# 更新原始 DataFrame
df.update(filtered_rows)
df

In [ ]:
# 替换 'end_date' 列中的异常值为指定的值，例如 'N/A'
df['end_date'].replace(to_replace='^9999.*', value="NaN", regex=True, inplace=True)
# titanic["Sex_short"] = titanic["Sex"].replace({"male": "M", "female": "F"})
# 但它提供了一种使用映射或词汇表来转换某些值的便捷方法。它需要dictionary来定义映射。{from : to}
df

In [ ]:
# 将 'end_date' 列中的 NaN 替换为默认值（例如，0）
df['start_date'].fillna(date.today(), inplace=True)
# fillna 不会直接处理NAT。 需要单独赋值操作
df['end_date'] = df['end_date'].fillna(date.today())
df

In [ ]:
# 将 'start_date' 和 'end_date' 列中的值转换为 datetime 类型
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['start_date'].min(), df['start_date'].max(), df['end_date'].min(), df['end_date'].max()

In [ ]:
from datetime import timedelta
import numpy as np

start_date = date(2022, 1, 1)
end_date = date(2022, 12, 31)
df['max_start_date'] = np.maximum(df['start_date'].dt.date, start_date)
df['min_end_date'] = np.minimum(df['end_date'].dt.date, end_date)
df['work_days'] = np.minimum(df['end_date'].dt.date, end_date) - np.maximum(df['start_date'].dt.date, start_date)
# df[df['work_days'] > timedelta(days=180)]
df = df[np.minimum(df['end_date'].dt.date, end_date) - np.maximum(df['start_date'].dt.date, start_date) > timedelta(
    days=180)]
df[['work_days', 'max_start_date', 'min_end_date']]

In [ ]:
grouped_df = df.groupby(["people_name", "comp_name_zh_cn"])
grouped_df

In [ ]:
grouped_df = grouped_df.agg({"employer": "count"})
grouped_df

In [ ]:
sorted_df = grouped_df.sort_values(by="employer", ascending=False)
sorted_df = sorted_df[sorted_df['employer'] > 3]
sorted_df